In [2]:
!pip install -U databento scikit-learn statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.3 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import databento as db
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm

#######################
# 0) Constants
#######################
API_KEY = "DATABENTO_API_KEY"
STOCKS = ["AAPL", "AMGN", "TSLA", "JPM", "XOM"]
DATASET = "XNAS.ITCH"          # Nasdaq TotalView-ITCH
SCHEMA = "mbp-10"              # MBP-10 data
START = "2024-01-01T00:00:00"
END   = "2024-01-07T23:59:59"
LEVELS = 5                     # Number of LOB levels to process
OUT_DATA_DIR = "../data"       # directory to save final pickled data
OUT_RESULTS_DIR = "../results" # directory to save charts, etc.

#######################
# 1) Ensure Folders
#######################
os.makedirs(OUT_DATA_DIR, exist_ok=True)
os.makedirs(OUT_RESULTS_DIR, exist_ok=True)

#######################
# 2) Initialize Client
#######################
client = db.Historical(key=API_KEY)

#######################
# 3) Loop Over Stocks
#######################
for symbol in STOCKS:
    print(f"Processing {symbol} ...")

    # ------------------------------------
    # (A) Download tick-level MBP-10 data
    # ------------------------------------
    data = client.timeseries.get_range(
        dataset=DATASET,
        symbols=[symbol],
        schema=SCHEMA,
        start=START,
        end=END,
    )
    df = data.to_df()
    print(f"Raw data shape for {symbol}:", df.shape)

    # Basic cleanup
    df["ts_event"] = pd.to_datetime(df["ts_event"], unit="ns", errors="coerce")
    df = df.sort_values("ts_event").reset_index(drop=True)

    # (Optional) Filter out pre-/post-market if needed
    # e.g., if times are in UTC, adjust accordingly:
    # df = df[(df["ts_event"].dt.time >= pd.to_datetime("09:30").time()) &
    #         (df["ts_event"].dt.time <  pd.to_datetime("16:00").time())]

    # ------------------------------------
    # (B) Compute Multi-Level OFI at the event level
    # ------------------------------------
    for lvl in range(LEVELS):
        bsz_col = f"bid_sz_0{lvl}"
        asz_col = f"ask_sz_0{lvl}"
        df[bsz_col] = df[bsz_col].fillna(0)
        df[asz_col] = df[asz_col].fillna(0)
        # Event-level incremental OFI
        df[f"OFI_{lvl}"] = df[bsz_col].diff().fillna(0) - df[asz_col].diff().fillna(0)

    # ------------------------------------
    # (C) Resample to 1-minute
    # ------------------------------------
    df = df.set_index("ts_event")
    df_1m = pd.DataFrame()

    # Sum OFI increments, last top-of-book for mid_price
    for lvl in range(LEVELS):
        ofi_col = f"OFI_{lvl}"
        df_1m[ofi_col] = df[ofi_col].resample("1min").sum()

    # Keep track of best-level px to define mid_price
    df_1m["bid_px_00"] = df["bid_px_00"].resample("1min").last()
    df_1m["ask_px_00"] = df["ask_px_00"].resample("1min").last()

    # Optionally keep best-level sizes
    df_1m["bid_sz_00"] = df["bid_sz_00"].resample("1min").last()
    df_1m["ask_sz_00"] = df["ask_sz_00"].resample("1min").last()

    # Drop intervals with no data
    df_1m = df_1m.dropna(how="all")

    # ------------------------------------
    # (D) Mid-price & 1-minute returns
    # ------------------------------------
    df_1m["mid_price"] = (df_1m["bid_px_00"] + df_1m["ask_px_00"]) / 2
    df_1m["mid_price"] = df_1m["mid_price"].ffill()
    df_1m["log_ret"] = np.log(df_1m["mid_price"].shift(-1) / df_1m["mid_price"])

    # Exclude first/last 30 minutes if needed
    # df_1m = df_1m.between_time("10:00", "15:30")

    # ------------------------------------
    # (E) PCA Integration
    # ------------------------------------
    ofi_cols = [f"OFI_{lvl}" for lvl in range(LEVELS)]
    X = df_1m[ofi_cols].fillna(0).values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=1)
    ofi_pca = pca.fit_transform(X_scaled)
    df_1m["OFI_PCA"] = ofi_pca

    var_ratio = pca.explained_variance_ratio_[0] if len(pca.explained_variance_ratio_) > 0 else None
    print(f"{symbol}: PCA 1st component explains {100*var_ratio:.2f}% of variance") if var_ratio else None

    # ------------------------------------
    # (F) Quick single-stock regression example
    # ------------------------------------
    df_1m["OFI_PCA_lag1"] = df_1m["OFI_PCA"].shift(1)
    reg_data = df_1m.dropna(subset=["log_ret", "OFI_PCA_lag1"])
    if len(reg_data) > 5:  # Need enough data points
        X = sm.add_constant(reg_data["OFI_PCA_lag1"])
        y = reg_data["log_ret"]
        model = sm.OLS(y, X)
        results = model.fit()
        print(f"\nSingle-stock OLS for {symbol} (1-min lag):")
        print(results.summary())
    else:
        print(f"\nNot enough data points for {symbol} regression.")

    # ------------------------------------
    # (G) Save final DataFrame
    # ------------------------------------
    output_path = os.path.join(OUT_DATA_DIR, f"{symbol}_1m_aggregated.pkl")
    df_1m.to_pickle(output_path)
    print(f"Saved {symbol} final DataFrame to {output_path}\n")

Processing AAPL ...
Raw data shape for AAPL: (9587204, 73)
AAPL: PCA 1st component explains 32.16% of variance

Single-stock OLS for AAPL (1-min lag):
                            OLS Regression Results                            
Dep. Variable:                log_ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.4267
Date:                Mon, 10 Feb 2025   Prob (F-statistic):              0.514
Time:                        11:36:32   Log-Likelihood:                 33731.
No. Observations:                5218   AIC:                        -6.746e+04
Df Residuals:                    5216   BIC:                        -6.745e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|   

In [4]:
df_1m.head() # Sample df for visualization purposes

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,OFI_0,OFI_1,OFI_2,OFI_3,OFI_4,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,mid_price,log_ret,OFI_PCA,OFI_PCA_lag1
ts_event,,,,,,,,,,,,,
2024-01-02 09:00:00+00:00,-4.294966e+09,4.724464e+10,4.294967e+10,3.865471e+10,4.294967e+10,100.70,101.0,1.0,27.0,100.850,0.000694,1.172401,NaN
2024-01-02 09:01:00+00:00,4.100000e+01,4.294967e+10,3.006477e+10,3.435974e+10,2.576980e+10,100.84,101.0,16.0,1.0,100.920,0.000050,0.944716,1.172401
2024-01-02 09:02:00+00:00,4.294967e+09,4.294967e+09,-3.865471e+10,-3.435974e+10,-8.589934e+09,100.85,101.0,16.0,1.0,100.925,-0.000743,-0.234113,0.944716
2024-01-02 09:03:00+00:00,1.288490e+10,6.012954e+10,2.576980e+10,2.147484e+10,6.012954e+10,100.70,101.0,1.0,1.0,100.850,0.000545,1.179117,-0.234113
2024-01-02 09:04:00+00:00,-1.288490e+10,-2.576980e+10,-1.717987e+10,5.583457e+10,5.153961e+10,100.81,101.0,17.0,1.0,100.905,0.000396,0.942060,1.179117
